DNN testing playground! 

i wanna put all the final shit for the DNN into a .py file but for now while I fuck with stuff, ill do it here. that is if this bad boi can even handle the big ass .h5py files I made, if not either make smaller ones or just use alpha.

In [14]:
# boilerplate, ensure you have these packages
import numpy as np
import tensorflow as tf
from keras import callbacks
from keras.callbacks import LearningRateScheduler as LRS
from keras.models import Model
from keras.layers import *
from keras import optimizers as kopt
import keras.backend as K
import datetime
import h5py
import hyperopt as hopt

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rc('xtick', direction='in', top=True)
mpl.rc('ytick', direction='in', right=True)
mpl.rc('xtick.minor', visible=True)
mpl.rc('ytick.minor', visible=True)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9246477179882743693
]


In [2]:
# data rescaling functions, want everything to be [0,1]
# rescaling thicknesses, angle of incid, psi, delta
# this info should be pulled from the corresponding .txt file!!!
# ^although im not quite sure how I wanna do tht rn so just list it in the .txt file
def resc_l(thickness):
    'rescales the thicknesses to [0,1], takes in arrays'
    resc = thickness*1e7 
    return resc
def resc_ang(aoi):
    'rescales the aoi to [0,1], takes in arrays in DEGREES'
    resc = aoi/90.
    return resc
def resc_psi(psi):
    '''
    rescales ellipsometric coeff psi to basically [0,1], takes in array in DEGREES. returns in rescaled degrees
    '''
    resc = psi/90.
    return resc
def resc_del(delta):
    '''
    rescales ellipsometric coeff delta to be positive (by adding phase shift if needed)
    takes in array in RADIANS (b/c its the natural output of tmm.ellips). returns rescaled degrees
    '''
    if (delta.min()<0.):
        resc = delta + np.pi
    resc = np.rad2deg(resc)/360.
    return resc

In [46]:
# funct to read in data
def read_dat(fname,dataset,nlay,nang,nwvl):
    '''
    function to read in the training data within an .h5 file and rescale
    currently used for ENZ proj with data for angle, thicknesses, rp, rs, tp, ts, psi, delta
    NOTE: data should be a [N,T] array, w/ N for num of runs and T = nang + nlay + 6*nang*nwvl
    inputs:
    fname - string - file name containing the training data, pls use full file name
    dataset - string - name of the dataset in the .h5 file
    NOTE: currently doesn't support reading in data from more than 1 dataset
    nlay - integer - number of layers in the multistack
    nang - integer - number of angles of incidence investigated
    nwvl - integer - number of wavelengths investigated
    NOTE: will throw errors if inputs are not correct
    outputs:
    ang - array, float - [N,nang] array of rescaled angles
    l - array, float - [N,nlay] array of rescaled thicknesses
    rp,rs,tp,ts - array, float - each is a [N,nang*nwvl] array of R and T params
    psi, delta - array, float - each is a [N,nang*nwvl] array of ellipsometric coeff in degrees
    '''
    # assertions to make sure inputs are good
    assert isinstance(nang,int), 'incorrect input for: nang'
    assert isinstance(nlay,int), 'incorrect input for: nlay'
    assert isinstance(nwvl,int), 'incorrect input for: nwvl'
    assert isinstance(fname,str), 'incorrect input for: fname'
    assert isinstance(dataset,str), 'incorrect input for: dataset'
    with h5py.File(fname,'r') as h:
        dat = np.array(h.get(dataset))
    # assertion to make sure this will read in data correctly
    assert (dat[0,0,:].size == nang+nlay+6*nang*nwvl), 'total data shape and input shape do not match'
    ang = dat[:,0,0:nang]
    l = dat[:,0,nang:(nang+nlay)]
    high = nang+nlay+nang*nwvl
    rp = dat[:,0,(nang+nlay):high]
    low = high
    high += nang*nwvl
    rs = dat[:,0,low:high]
    low = high
    high += nang*nwvl
    tp = dat[:,0,low:high]
    low = high
    high += nang*nwvl
    ts = dat[:,0,low:high]
    low = high
    high += nang*nwvl
    psi = dat[:,0,low:high]
    low = high
    high += nang*nwvl
    delta = dat[:,0,low:high]
    # assertion to make sure data was read in correctly
    assert (dat[-1,0,-1]==delta[-1,-1]), 'data was read incorrectly'
    # delete the original array that was read in as it is just a space waster now
    del dat
    l = resc_l(l)
    ang = resc_ang(ang)
    psi = resc_psi(psi)
    # to make sure phase shift is introduced correctly, need to seperate delta by angle
    if (nang>1):
        for i in range(nang):
            delta[:,(i*nwvl):((i+1)*nwvl)] = resc_del(delta[:,(i*nwvl):((i+1)*nwvl)])
    delta = resc_del(delta)
    return (ang,l,rp,rs,tp,ts,psi,delta)

In [47]:
# function to split btwn training, validation, and testing datasets
# in the future may want this funct to just split the data itself either hardcoded in the funct or just default inputs
def split_dat(ang,l,rp,rs,tp,ts,psi,delta,n_tr,n_v,n_te):
    '''
    function that takes the newly read in data from the .h5 file and splits it into
    training, validation, and testing datasets
    note: want 90% of the data to be for training and 5% each for validation and testing
    note: currently assuming you only have one .h5 file and want to split that btwn the three
    inputs:
    ang,l,rp,rs,tp,ts,psi,delta - array, float - arrays with the data
    n_tr - integer - number of sets for training
    n_v - integer - number of sets for validation
    n_te - integer - number of sets for testing
    outputs:
    3x{ang,l,rp,rs,tp,ts,psi,delta} - array, float - arrays with data for each purpose
    '''
    # make sure you are able to split the data into the amounts you intended
    assert (len(ang) == n_tr+n_v+n_te), 'error: number of dataset mismatch'
    # training
    trang=ang[:n_tr,:]; trl=l[:n_tr,:]; trrp=rp[:n_tr,:]; trrs=rs[:n_tr,:]; trtp=tp[:n_tr,:]; trts=ts[:n_tr,:]; trpsi=psi[:n_tr,:]; trdel=delta[:n_tr,:]
    low = n_tr
    high = n_tr + n_v
    # validation
    vang=ang[low:high,:]; vl=l[low:high,:]; vrp=rp[low:high,:]; vrs=rs[low:high,:]; vtp=tp[low:high,:]; vts=ts[low:high,:]; vpsi=psi[low:high,:]; vdel=delta[low:high,:]
    low = high
    high += n_te
    # testing
    teang=ang[low:high,:]; tel=l[low:high,:]; terp=rp[low:high,:]; ters=rs[low:high,:]; tetp=tp[low:high,:]; tets=ts[low:high,:]; tepsi=psi[low:high,:]; tedel=delta[low:high,:]
    return (trang,trl,trrp,trrs,trtp,trts,trpsi,trdel,vang,vl,vrp,vrs,vtp,vts,vpsi,vdel,teang,tel,terp,ters,tetp,tets,tepsi,tedel)    

In [48]:
# function that combines 'read_dat' and 'split_dat'
def rs_dat(fname,dataset,nlay,nang,nwvl,n_tr,n_v,n_te):
    '''
    function to take care of both 'read_dat' and 'split dat'
    and should throw errors if things go wrong
    inputs:
    fname - string - file name containing the training data, pls use full file name
    dataset - string - name of the dataset in the .h5 file
    NOTE: currently doesn't support reading in data from more than 1 dataset
    nlay - integer - number of layers in the multistack
    nang - integer - number of angles of incidence investigated
    nwvl - integer - number of wavelengths investigated
    n_tr - integer - number of sets for training
    n_v - integer - number of sets for validation
    n_te - integer - number of sets for testing
    outputs:
    3x{ang,l,rp,rs,tp,ts,psi,delta} - array, float - arrays with data for each purpose
    NOTE: make sure you have enough arrays rdy as output bc this funct deletes redundant data
    ^ its 24 (3x8) arrays btw
    '''
    (a,b,c,d,e,f,g,h) = read_dat(fname,dataset,nlay,nang,nwvl)
    (trang,trl,trrp,trrs,trtp,trts,trpsi,trdel,vang,vl,vrp,vrs,vtp,vts,vpsi,vdel,teang,tel,terp,ters,tetp,tets,tepsi,tedel) = split_dat(a,b,c,d,e,f,g,h,n_tr,n_v,n_te)
    del a; del b; del c; del d; del e; del f; del g; del h
    return (trang,trl,trrp,trrs,trtp,trts,trpsi,trdel,vang,vl,vrp,vrs,vtp,vts,vpsi,vdel,teang,tel,terp,ters,tetp,tets,tepsi,tedel)


In [16]:
# learning rate decay function
def decay(ep):
    'learning rate decay function, input is epochs, may not actually be used since i dont have live update guy'
    a = .0025
    lr = a/((ep)+1)
    return lr
lr = LRS(decay)

In [18]:
# heres the model babey
def build_mod(drop,nwvl,act='relu'):
    '''
    builds a DNN model that can be used for machine learing
    currently takes the inputs based by angle, does the DNN thing and should be outputting
    the thicknesses of each layer in the multistack.
    currently assuming there are three aoi's
    input:
    drop - float - dropout rate can be (0,1) exclusive
    nwvl - integer - number of wavelengths investigated
    act - string - activation method for Dense layer, default 'relu', see keras Dense documentation for more options
    ^ NOTE: this is for the inner Dense layers the ones at the end can/will be different
    
    output:
    model - model obj - a full fledged keras model, it has ONLY BEEN MADE, needs to be compiled and named
    '''
    # inputs are grouped by aoi
    a1 = Input((1,),name='angle_1')
    rp1 = Input((nwvl,),name='rp_1'); rs1 = Input((nwvl,),name='rs_1')
    tp1 = Input((nwvl,),name='tp_1'); ts1 = Input((nwvl,),name='ts_1')
    p1 = Input((nwvl,),name='psi_1'); d1 = Input((nwvl,),name='delta_1')
    a2 = Input((1,),name='angle_2')
    rp2 = Input((nwvl,),name='rp_2'); rs2 = Input((nwvl,),name='rs_2')
    tp2 = Input((nwvl,),name='tp_2'); ts2 = Input((nwvl,),name='ts_2')
    p2 = Input((nwvl,),name='psi_2'); d2 = Input((nwvl,),name='delta_2')
    a3 = Input((1,),name='angle_3')
    rp3 = Input((nwvl,),name='rp_3'); rs3 = Input((nwvl,),name='rs_3')
    tp3 = Input((nwvl,),name='tp_3'); ts3 = Input((nwvl,),name='ts_3')
    p3 = Input((nwvl,),name='psi_3'); d3 = Input((nwvl,),name='delta_3')
    # connect the inputs
    # currently im thinking connecting them in a similar fasion as the rows
    # these are midpoints and so shall be named: m##, 1st #: layer num, 2nd #: index within layer
    # note: add layer makes a tensor of the same size of the inputs but idk how tht works if they arent the same size so im just doing it like andy for rn
    m11 = Add()([rp1,rs1,a1]); m12 = Add()([tp1,ts1,a1]); m13 = Add()([p1,d1,a1])
    m14 = Add()([rp2,rs2,a2]); m15 = Add()([tp2,ts2,a2]); m16 = Add()([p2,d2,a2])
    m17 = Add()([rp3,rs3,a3]); m18 = Add()([tp3,ts3,a3]); m19 = Add()([p3,d3,a3])
    # dense layer, followed by a dropout
    # name scheme: x##: 1st #: dense layer num, 2nd #: index within layer
    x11 = Dense(nwvl,activation=act)(m11); x11 = Dropout(drop)(x11)
    x12 = Dense(nwvl,activation=act)(m12); x12 = Dropout(drop)(x12)
    x13 = Dense(nwvl,activation=act)(m13); x13 = Dropout(drop)(x13)
    x14 = Dense(nwvl,activation=act)(m14); x14 = Dropout(drop)(x14)
    x15 = Dense(nwvl,activation=act)(m15); x15 = Dropout(drop)(x15)
    x16 = Dense(nwvl,activation=act)(m16); x16 = Dropout(drop)(x16)
    x17 = Dense(nwvl,activation=act)(m17); x17 = Dropout(drop)(x17)
    x18 = Dense(nwvl,activation=act)(m18); x18 = Dropout(drop)(x18)
    x19 = Dense(nwvl,activation=act)(m19); x19 = Dropout(drop)(x19)
    # another Add layer, now m2#, combining all data from each aoi together
    m21 = Add()([x11,x12,x13])
    m22 = Add()([x14,x15,x16])
    m23 = Add()([x17,x18,x19])
    # another Dense layer, also followed by a dropout 
    # name now x2#, also increase output size
    x21 = Dense((nwvl+50),activation=act)(m21); x21 = Dropout(drop)(x21)
    x22 = Dense((nwvl+50),activation=act)(m22); x22 = Dropout(drop)(x22)
    x23 = Dense((nwvl+50),activation=act)(m23); x23 = Dropout(drop)(x23)
    # another Add layer, now m3#, combining all data together
    m31 = Add()([x21,x22,x23])
    # another dense layer with dropout after, now x3#
    x31 = Dense((nwvl+100),activation=act)(m31)
    x31 = Dropout(drop)(x31)
    # last 'internal' layer, prep for output layer
    # note: i have no idea what the output for this layer should be, consult andy
    x41 = Dense(100,activation=act)(x31)
    # output layer
    # note: no idea what the activation should be for this one, consult andy
    l_out = Dense(15,activation=None)(x41)
    # put it all together and compile
    model = Model([a1,rp1,rs1,tp1,ts1,p1,d1,a2,rp2,rs2,tp2,ts2,p2,d2,a3,rp3,rs3,tp3,ts3,p3,d3],[l_out])
    return model

In [34]:
drop = 0.
wvl = np.linspace(400.,1000.,121)
nwvl = len(wvl)

In [24]:
mod = build_mod(drop,nwvl)
# compiling the same way as andy rn
mod.compile(optimizer='adam',loss=['mse'],metrics=['mse'],loss_weights=[10])
modname = 'enzchar_invdes_model_d21621.h5'

In [25]:
mod.save(modname)

In [27]:
# you beautiful son of a bitch, naming the input layers means you can pass in a dictionary!!
# do model stuff here


In [28]:
# testing all the shit
import run_data

Will parallelize with 1 cores.

doing that data thang, please wait...
puttin that mf in ya file
congrats u done!


In [30]:
with h5py.File('traindata_enzchar_lay15_mats3_n0.5k_20210216.hdf5','r') as h:
    dat = np.array(h.get('data'))


In [45]:
dat[:,0,:].shape
# 3+15+6*3*121

(500, 2196)

In [49]:
(trang,trl,trrp,trrs,trtp,trts,trpsi,trdel,vang,vl,vrp,vrs,vtp,vts,vpsi,vdel,teang,tel,terp,ters,tetp,tets,tepsi,tedel) = rs_dat('traindata_enzchar_lay15_mats3_n0.5k_20210216.hdf5','data',15,3,nwvl,400,50,50)